<a href="https://colab.research.google.com/github/alohia/pytorch_playground/blob/master/01_pytorch_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tensors and Tensor Operations

In [0]:
import torch
from torch import tensor

In [3]:
a = tensor([10., 6, -3])
b = tensor([2., 7, 4])
a, b

(tensor([10.,  6., -3.]), tensor([2., 7., 4.]))

In [4]:
a+b

tensor([12., 13.,  1.])

In [6]:
a < b

tensor([False,  True,  True])

In [0]:
preds = tensor([1, 0, 0, 1])
y = tensor([1, 1, 0, 0])

In [8]:
(preds==y).float().mean()

tensor(0.5000)

In [9]:
m = tensor([[1., 2, 3], [4, 5, 6], [7, 8, 9]])
m

tensor([[1., 2., 3.],
        [4., 5., 6.],
        [7., 8., 9.]])

In [10]:
m.shape

torch.Size([3, 3])

In [12]:
# Broadcasting with a scalar
a, a + 2

(tensor([10.,  6., -3.]), tensor([12.,  8., -1.]))

In [14]:
m*2

tensor([[ 2.,  4.,  6.],
        [ 8., 10., 12.],
        [14., 16., 18.]])

## Broadcast vector to matrix

In [15]:
c = tensor([10., 20, 30]); c

tensor([10., 20., 30.])

In [16]:
m

tensor([[1., 2., 3.],
        [4., 5., 6.],
        [7., 8., 9.]])

In [17]:
m.shape, c.shape

(torch.Size([3, 3]), torch.Size([3]))

In [18]:
m+c

tensor([[11., 22., 33.],
        [14., 25., 36.],
        [17., 28., 39.]])

In [19]:
c+m

tensor([[11., 22., 33.],
        [14., 25., 36.],
        [17., 28., 39.]])

In [0]:
# See what happens when broadcasting. It's not a real copy - but looks like so

t = c.expand_as(m)

In [21]:
t

tensor([[10., 20., 30.],
        [10., 20., 30.],
        [10., 20., 30.]])

In [22]:
m+t

tensor([[11., 22., 33.],
        [14., 25., 36.],
        [17., 28., 39.]])

In [23]:
t.storage() #only storing one copy

 10.0
 20.0
 30.0
[torch.FloatStorage of size 3]

In [33]:
c

tensor([10., 20., 30.])

In [31]:
c.unsqueeze(0)

tensor([[10., 20., 30.]])

In [29]:
c.unsqueeze(1)

(tensor([[10.],
         [20.],
         [30.]]), torch.Size([3]))

In [32]:
c.shape, c.unsqueeze(0).shape, c.unsqueeze(1).shape

(torch.Size([3]), torch.Size([1, 3]), torch.Size([3, 1]))

In [34]:
c[None, :]

tensor([[10., 20., 30.]])

In [35]:
c[:, None].expand_as(m)

tensor([[10., 10., 10.],
        [20., 20., 20.],
        [30., 30., 30.]])

In [37]:
m + c[:, None]

tensor([[11., 12., 13.],
        [24., 25., 26.],
        [37., 38., 39.]])

In [48]:
m

tensor([[1., 2., 3.],
        [4., 5., 6.],
        [7., 8., 9.]])

In [47]:
m.sum(0), m.sum(1)

(tensor([12., 15., 18.]), tensor([ 6., 15., 24.]))

# Matrix Multiplication

In [0]:
m1 = torch.randn(5, 20)
m2 = torch.randn(20, 3)

In [53]:
m1.shape, m2.shape

(torch.Size([5, 20]), torch.Size([20, 3]))

In [0]:
def matmul(a,b):
    ar, ac = a.shape # n_rows * n_cols
    br, bc = b.shape
    assert ac==br
    c = torch.zeros(ar, bc)
    for i in range(ar):
        for j in range(bc):
            for k in range(ac): # or br
                c[i,j] += a[i,k] * b[k,j]
    return c

In [55]:
%time t1=matmul(m1, m2)

CPU times: user 10.8 ms, sys: 0 ns, total: 10.8 ms
Wall time: 12 ms


In [56]:
t1.shape

torch.Size([5, 3])

### Matmul with broadcasting

In [0]:
def matmul(a,b):
    ar, ac = a.shape
    br, bc = b.shape
    assert ac==br
    c = torch.zeros(ar, bc)
    for i in range(ar):
        c[i] = (a[i].unsqueeze(-1)*b).sum(dim=0)
    return c

In [58]:
%time t2=matmul(m1, m2)

CPU times: user 1.24 ms, sys: 223 µs, total: 1.47 ms
Wall time: 1.46 ms


In [59]:
t2.shape

torch.Size([5, 3])

In [61]:
torch.allclose(t1, t2)

True

### Pytorch built-in

In [66]:
%timeit -n 10 t3 = m1.matmul(m2)

The slowest run took 11.94 times longer than the fastest. This could mean that an intermediate result is being cached.
10 loops, best of 3: 8.12 µs per loop


In [71]:
%timeit -n 10 t4 = m1@m2

The slowest run took 36.64 times longer than the fastest. This could mean that an intermediate result is being cached.
10 loops, best of 3: 8.44 µs per loop


In [72]:
torch.allclose(t3, t4)

True